In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd
import joblib
import streamlit as st  # Pour l'application Streamlit

# Imports TensorFlow/Keras
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow as tf

# Imports Scikit-learn
from sklearn import manifold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA


from tqdm import tqdm

# --- Constantes de configuration ---
PHOTOS_JSON_PATH = 'data/photos.json'
PHOTOS_DIR_PATH = 'data/photos/'
N_SAMPLES_PER_CLASS = 500
TARGET_COLUMN = 'label'
EFFICIENTNET_INPUT_SIZE = (224, 224)
PCA_N_COMPONENTS = 10
TEST_SIZE_SPLIT = 0.33
RANDOM_STATE_GLOBAL = 42

# Paths for saving/loading models
SCALER_PATH = 'scaler.joblib'
PCA_PATH = 'pca.joblib'
MODEL_PATH = 'random_forest_model.joblib'
LABEL_ENCODER_PATH = 'label_encoder.joblib'
TSNE_PLOT_IMAGE_PATH = 'tsne_visualization.png'
FEATURE_EXTRACTOR_PATH = 'feature_extractor.joblib' # Pour sauvegarder EfficientNetB0


# --- 1. Chargement et préparation des données ---
print("1. Chargement et préparation des données...")

def load_data_from_jsonl(file_path):
    """Charge les données depuis un fichier JSON Lines.

    Args:
        file_path (str): Le chemin du fichier JSON Lines.

    Returns:
        pd.DataFrame: Les données chargées, ou un DataFrame vide en cas d'erreur.
    """
    data_list = []
    try:
        with open(file_path, 'r') as f:
            for line in f:
                data_list.append(json.loads(line))
    except FileNotFoundError:
        print(f"Erreur : Le fichier {file_path} n'a pas été trouvé.")
        return pd.DataFrame()
    except json.JSONDecodeError as e:
        print(f"Erreur de décodage JSON dans {file_path}: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"Une erreur inattendue s'est produite lors du chargement de {file_path}: {e}")
        return pd.DataFrame()

    return pd.DataFrame(data_list)


df_full = load_data_from_jsonl(PHOTOS_JSON_PATH)

if df_full.empty:
    print("Aucune donnée chargée. Arrêt du script.")
    exit()


def stratified_sample_df(df: pd.DataFrame, col: str, n_samples: int) -> pd.DataFrame:
    """
    Effectue un échantillonnage stratifié sur un DataFrame.
    S'assure que chaque classe a au moins n_samples, sinon prend le minimum disponible.

    Args:
        df (pd.DataFrame): Le DataFrame à échantillonner.
        col (str): La colonne à utiliser pour la stratification.
        n_samples (int): Le nombre d'échantillons souhaité par classe.

    Returns:
        pd.DataFrame: Un DataFrame échantillonné.
    """
    min_samples_in_any_class = df[col].value_counts().min()
    actual_n_samples = min(n_samples, min_samples_in_any_class)
    if actual_n_samples < n_samples:
        print(f"Avertissement : Le nombre d'échantillons demandé ({n_samples}) est supérieur au nombre "
              f"d'échantillons disponibles dans la plus petite classe ({min_samples_in_any_class}). "
              f"Utilisation de {actual_n_samples} échantillons par classe.")

    try:
        df_sampled = df.groupby(col, group_keys=False).apply(
            lambda x: x.sample(n=actual_n_samples, random_state=RANDOM_STATE_GLOBAL))
        return df_sampled.reset_index(drop=True)
    except KeyError:
        print(f"Erreur : La colonne '{col}' n'existe pas dans le DataFrame.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'échantillonnage stratifié : {e}")
        return pd.DataFrame()



subsampled_df = stratified_sample_df(df_full, TARGET_COLUMN, N_SAMPLES_PER_CLASS)
if subsampled_df.empty:
    print("Aucun échantillon stratifié n'a pu être créé. Arrêt du script.")
    exit()
print(f"Nombre d'échantillons après sous-échantillonnage stratifié : {len(subsampled_df)}")
print(f"Distribution des classes après échantillonnage :\n{subsampled_df[TARGET_COLUMN].value_counts()}")

# --- 2. Extraction des caractéristiques (Features) ---
print("\n2. Extraction des caractéristiques avec EfficientNetB0...")

# Chargement du modèle pré-entraîné EfficientNetB0 pour l'extraction de caractéristiques
try:
    feature_extractor = EfficientNetB0(include_top=False, weights='imagenet', pooling='avg')
except Exception as e:
    print(f"Erreur lors du chargement du modèle EfficientNetB0 : {e}")
    exit()

def extract_features(image_path, model, target_size):
    """
    Extrait les caractéristiques d'une image avec un modèle donné.

    Args:
        image_path (str): Chemin vers l'image.
        model: Modèle Keras pour l'extraction des caractéristiques.
        target_size (tuple): Taille de l'image cible.

    Returns:
        np.ndarray: Vecteur de caractéristiques extrait, ou None en cas d'erreur.
    """
    try:
        image_array = cv2.imread(image_path)
        if image_array is None:
            raise ValueError(f"Impossible de charger l'image depuis {image_path}")

        img_rgb = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, target_size)
        img_expanded = np.expand_dims(img_resized, axis=0)
        img_preprocessed = preprocess_input(img_expanded)

        return model.predict(img_preprocessed, verbose=0)[0]
    except Exception as e:
        print(f"Erreur lors de l'extraction des caractéristiques de l'image {image_path}: {e}")
        return None
    
processed_photo_ids = []
processed_features_X = []
processed_labels_y = []

for index, row in tqdm(subsampled_df.iterrows(), total=subsampled_df.shape[0], desc="Extraction des Features"):
    photo_id = row['photo_id']
    label = row[TARGET_COLUMN]
    image_path = os.path.join(PHOTOS_DIR_PATH, photo_id + '.jpg')

    feature_vector = extract_features(image_path, feature_extractor, EFFICIENTNET_INPUT_SIZE)
    if feature_vector is not None:
        processed_photo_ids.append(photo_id)
        processed_features_X.append(feature_vector)
        processed_labels_y.append(label)

if not processed_features_X:
    print("Erreur critique : Aucune caractéristique n'a été extraite avec succès. Le script va s'arrêter.")
    exit(1)

# Conversion en tableaux NumPy pour scikit-learn
features_X_np = np.stack(processed_features_X)
labels_y_np = np.array(processed_labels_y)
print(f"\n{len(features_X_np)} caractéristiques extraites avec succès.")

# Initialisation du LabelEncoder.
label_encoder = None
# Encodage des étiquettes si elles sont de type chaîne de caractères
if labels_y_np.ndim > 0 and labels_y_np.dtype.kind in ['O', 'S', 'U']:
    print("Encodage des étiquettes (labels) car elles sont de type chaîne de caractères.")
    label_encoder = LabelEncoder()
    labels_y_for_model = label_encoder.fit_transform(labels_y_np)
    label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    print(f"Mappage des étiquettes : {label_mapping}")
else:
    labels_y_for_model = labels_y_np

# --- 3. Réduction de dimensionnalité avec t-SNE et Visualisation ---
print("\n3. Réduction de dimensionnalité avec t-SNE et Visualisation...")
# Vérification que features_X_np a au moins 2 éléments
if len(features_X_np) < 2:
    print("Pas assez d'échantillons pour appliquer t-SNE. La visualisation sera ignorée.")
else:
    # Ajustement de la perplexité pour t-SNE si le nombre d'échantillons est faible
    perplexity_value = min(30, len(features_X_np) - 1)
    try:
        tsne = manifold.TSNE(n_components=2, init='pca', random_state=RANDOM_STATE_GLOBAL,
                               perplexity=perplexity_value)
        X_tsne = tsne.fit_transform(features_X_np)

        plt.figure(figsize=(10, 8))
        sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=labels_y_np, palette="viridis")
        plt.title('Visualisation t-SNE des caractéristiques des images')
        plt.xlabel('t-SNE Composante 1')
        plt.ylabel('t-SNE Composante 2')
        plt.legend(title=TARGET_COLUMN)
        plt.show()

        plt.savefig(TSNE_PLOT_IMAGE_PATH)
        plt.close()
        print(f"Visualisation t-SNE sauvegardée dans {TSNE_PLOT_IMAGE_PATH}")
    except Exception as e:
        print(f"Erreur lors de l'exécution de t-SNE ou de la visualisation : {e}")

# --- 4. Apprentissage d'un classifieur ---
print("\n4. Apprentissage d'un classifieur Random Forest...")

# Division des données (en utilisant les features et labels correctement alignés)
X_train, X_test, y_train, y_test = train_test_split(
    features_X_np, labels_y_for_model,
    test_size=TEST_SIZE_SPLIT,
    random_state=RANDOM_STATE_GLOBAL,
    stratify=labels_y_for_model
)

print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]}, Test : {X_test.shape[0]}")

# Standardisation des données
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Réduction de dimensionnalité avec PCA
print(f"Application de la PCA avec {PCA_N_COMPONENTS} composantes...")
actual_pca_n_components = min(PCA_N_COMPONENTS, X_train_std.shape[1], X_train_std.shape[0])
if actual_pca_n_components < 1:
    actual_pca_n_components = 1
try:
    pca = PCA(n_components=actual_pca_n_components)
    X_train_pca = pca.fit_transform(X_train_std)
    X_test_pca = pca.transform(X_test_std)
    print(f"Variance expliquée par les {pca.n_components_} composantes PCA : {np.sum(pca.explained_variance_ratio_):.2f}")
except Exception as e:
    print(f"Erreur lors de l'application de PCA : {e}")
    X_train_pca = X_train_std
    X_test_pca = X_test_std
    pca = None

# Recherche des meilleurs hyperparamètres pour RandomForestClassifier avec GridSearchCV
print("Recherche des meilleurs hyperparamètres pour RandomForestClassifier...")
params_rf = {'max_depth': range(5, 20, 3), 'n_estimators': [100, 200, 300]}
rf_classifier_model = RandomForestClassifier(random_state=RANDOM_STATE_GLOBAL)

cv_folds = 3
if len(np.unique(y_train)) > 1 and len(y_train) > 0:
    min_samples_per_class_train = np.min(np.bincount(y_train))
    cv_folds = min(cv_folds, min_samples_per_class_train)
if cv_folds < 2:
    cv_folds = 2
if X_train_pca.shape[0] < cv_folds:
    cv_folds = X_train_pca.shape[0]

try:
    grid_search_clf = GridSearchCV(rf_classifier_model, params_rf, cv=cv_folds, scoring='accuracy')
    grid_search_clf.fit(X_train_pca, y_train)

    print(f"Meilleurs hyperparamètres trouvés : {grid_search_clf.best_params_}")

    # Évaluation du meilleur modèle
    best_rf_model = grid_search_clf.best_estimator_
    score = best_rf_model.score(X_test_pca, y_test)
    print(f"Score du classifieur (Random Forest) sur l'ensemble de test : {score:.4f}")
except Exception as e:
    print(f"Erreur lors de la recherche des hyperparamètres ou de l'évaluation du modèle: {e}")
    best_rf_model = None

# --- 5. Sauvegarde des modèles et transformateurs entraînés ---
print("\nSauvegarde des modèles et transformateurs entraînés...")
try:
    joblib.dump(scaler, SCALER_PATH)
    print(f"Scaler sauvegardé dans {SCALER_PATH}")
    if pca is not None:
        joblib.dump(pca, PCA_PATH)
        print(f"PCA sauvegardé dans {PCA_PATH}")
    if best_rf_model is not None:
        joblib.dump(best_rf_model, MODEL_PATH)
        print(f"Modèle RandomForest sauvegardé dans {MODEL_PATH}")
    if label_encoder:
        joblib.dump(label_encoder, LABEL_ENCODER_PATH)
        print(f"LabelEncoder sauvegardé dans {LABEL_ENCODER_PATH}")
    joblib.dump(feature_extractor, FEATURE_EXTRACTOR_PATH) # Sauvegarde du modèle EfficientNet
    print(f"EfficientNetB0 sauvegardé dans {FEATURE_EXTRACTOR_PATH}")
    print("Sauvegarde terminée.")
except Exception as e:
    print(f"Erreur lors de la sauvegarde des modèles : {e}")



def creer_image_factice(chemin, taille):
    """Crée une image factice noire si elle n'existe pas déjà."""
    if not os.path.exists(chemin):
        os.makedirs(os.path.dirname(chemin), exist_ok=True)
        cv2.imwrite(chemin, np.zeros((taille[0], taille[1], 3), dtype=np.uint8))
        print(f"Image factice créée : {chemin}")
    return chemin

# --- 7. Fonction Principale ---
def main():
    """Fonction principale pour exécuter le pipeline d'extraction de caractéristiques et d'entraînement."""
    # --- 1. Chargement et préparation des données ---
    df_full = load_data_from_jsonl(PHOTOS_JSON_PATH)
    if df_full.empty:
        return

    subsampled_df = stratified_sample_df(df_full, TARGET_COLUMN, N_SAMPLES_PER_CLASS)
    if subsampled_df.empty:
        return

    print(f"Nombre d'échantillons après sous-échantillonnage stratifié : {len(subsampled_df)}")
    print(f"Distribution des classes après échantillonnage :\n{subsampled_df[TARGET_COLUMN].value_counts()}")

    # --- 2. Extraction des caractéristiques ---
    print("\n2. Extraction des caractéristiques avec EfficientNetB0...")
    try:
        feature_extractor = EfficientNetB0(include_top=False, weights='imagenet', pooling='avg')
    except Exception as e:
        print(f"Erreur lors du chargement du modèle EfficientNetB0 : {e}")
        return

    processed_photo_ids = []
    processed_features_X = []
    processed_labels_y = []

    for index, row in tqdm(subsampled_df.iterrows(), total=subsampled_df.shape[0],
                            desc="Extraction des caractéristiques"):
        photo_id = row['photo_id']
        label = row[TARGET_COLUMN]
        image_path = os.path.join(PHOTOS_DIR_PATH, photo_id + '.jpg')
        feature_vector = extract_features(image_path, feature_extractor, EFFICIENTNET_INPUT_SIZE)

        if feature_vector is not None:
            processed_photo_ids.append(photo_id)
            processed_features_X.append(feature_vector)
            processed_labels_y.append(label)

    if not processed_features_X:
        print("Erreur critique : Aucune caractéristique n'a été extraite. Arrêt.")
        return

    features_X_np = np.stack(processed_features_X)
    labels_y_np = np.array(processed_labels_y)
    print(f"\n{len(features_X_np)} caractéristiques extraites avec succès.")

    # --- 3. Réduction de dimensionnalité et visualisation ---
    print("\n3. Réduction de dimensionnalité avec t-SNE et visualisation...")
    if len(features_X_np) >= 2:
        perplexity_value = min(30, len(features_X_np) - 1)
        try:
            tsne = manifold.TSNE(n_components=2, init='pca', random_state=RANDOM_STATE_GLOBAL,
                                    perplexity=perplexity_value)
            X_tsne = tsne.fit_transform(features_X_np)

            plt.figure(figsize=(10, 8))
            sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=labels_y_np, palette="viridis")
            plt.title('Visualisation t-SNE des caractéristiques des images')
            plt.xlabel('t-SNE Composante 1')
            plt.ylabel('t-SNE Composante 2')
            plt.legend(title=TARGET_COLUMN)
            plt.show()
            plt.savefig(TSNE_PLOT_IMAGE_PATH)
            plt.close()
            print(f"Visualisation t-SNE sauvegardée dans {TSNE_PLOT_IMAGE_PATH}")
        except Exception as e:
            print(f"Erreur lors de la visualisation t-SNE : {e}")
    else:
        print("Nombre d'échantillons insuffisant pour t-SNE. Visualisation ignorée.")

    # --- 4. Apprentissage du classifieur ---
    print("\n4. Apprentissage d'un classifieur Random Forest...")
    label_encoder = None
    if labels_y_np.ndim > 0 and labels_y_np.dtype.kind in ['O', 'S', 'U']:
        label_encoder = LabelEncoder()
        labels_y_for_model = label_encoder.fit_transform(labels_y_np)
    else:
        labels_y_for_model = labels_y_np

    X_train, X_test, y_train, y_test = train_test_split(
        features_X_np, labels_y_for_model,
        test_size=TEST_SIZE_SPLIT,
        random_state=RANDOM_STATE_GLOBAL,
        stratify=labels_y_for_model
    )

    print(f"Taille de l'ensemble d'entraînement : {X_train.shape[0]}, Test : {X_test.shape[0]}")

    scaler = StandardScaler()
    X_train_std = scaler.fit_transform(X_train)
    X_test_std = scaler.transform(X_test)

    actual_pca_n_components = min(PCA_N_COMPONENTS, X_train_std.shape[1], X_train_std.shape[0])
    if actual_pca_n_components < 1:
        actual_pca_n_components = 1
    try:
        pca = PCA(n_components=actual_pca_n_components)
        X_train_pca = pca.fit_transform(X_train_std)
        X_test_pca = pca.transform(X_test_std)
        print(f"Variance expliquée : {np.sum(pca.explained_variance_ratio_):.2f}")
    except Exception as e:
        print(f"Erreur lors de l'application de PCA : {e}")
        X_train_pca = X_train_std
        X_test_pca = X_test_std
        pca = None

    params_rf = {'max_depth': range(5, 20, 3), 'n_estimators': [100, 200, 300]}
    rf_classifier_model = RandomForestClassifier(random_state=RANDOM_STATE_GLOBAL)
    cv_folds = 3
    if len(np.unique(y_train)) > 1 and len(y_train) > 0:
        min_samples_per_class_train = np.min(np.bincount(y_train))
        cv_folds = min(cv_folds, min_samples_per_class_train)
    if cv_folds < 2:
        cv_folds = 2
    if X_train_pca.shape[0] < cv_folds:
        cv_folds = X_train_pca.shape[0]

    try:
        grid_search_clf = GridSearchCV(rf_classifier_model, params_rf, cv=cv_folds,
                                            scoring='accuracy')
        grid_search_clf.fit(X_train_pca, y_train)
        print(f"Meilleurs paramètres : {grid_search_clf.best_params_}")
        best_rf_model = grid_search_clf.best_estimator_
        score = best_rf_model.score(X_test_pca, y_test)
        print(f"Score sur l'ensemble de test : {score:.4f}")
    except Exception as e:
        print(f"Erreur lors de l'entraînement du modèle : {e}")
        best_rf_model = None

    # --- 5. Sauvegarde des modèles ---
    print("\n5. Sauvegarde des modèles et des transformateurs...")
    try:
        joblib.dump(scaler, SCALER_PATH)
        print(f"Scaler sauvegardé : {SCALER_PATH}")
        if pca is not None:
            joblib.dump(pca, PCA_PATH)
            print(f"PCA sauvegardé : {PCA_PATH}")
        if best_rf_model is not None:
            joblib.dump(best_rf_model, MODEL_PATH)
            print(f"Modèle sauvegardé : {MODEL_PATH}")
        if label_encoder:
            joblib.dump(label_encoder, LABEL_ENCODER_PATH)
            print(f"LabelEncoder sauvegardé : {LABEL_ENCODER_PATH}")
        joblib.dump(feature_extractor, FEATURE_EXTRACTOR_PATH) # Save EfficientNet model
        print(f"EfficientNetB0 sauvegardé : {FEATURE_EXTRACTOR_PATH}")
        print("Sauvegarde terminée.")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde : {e}")
        return

    # --- 6. Exemple de prédiction ---
    print("\n6. Exemple de prédiction sur une nouvelle image...")
    dummy_new_image_path = creer_image_factice(os.path.join(PHOTOS_DIR_PATH, "dummy_new_image.jpg"),
                                            EFFICIENTNET_INPUT_SIZE)
    if os.path.exists(dummy_new_image_path):
        predicted_topic = predict_image_topic(dummy_new_image_path,
                                                feature_extractor,
                                                scaler,
                                                pca,
                                                best_rf_model,
                                                label_encoder)
        if predicted_topic:
            print(f"Topic prédit pour '{dummy_new_image_path}': {predicted_topic}")
    else:
        print(f"L'image de test {dummy_new_image_path} n'a pas pu être créée. Test de prédiction ignoré.")

: 